In [ ]:
# optimize min

In [124]:
import gzip
import numpy
from collections import defaultdict

def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

In [125]:
data = []
for l in readGz("assignment1/train.json.gz"):
    data.append(l)

In [126]:
import random

# Randomly re-shuffle
# numpy.random.shuffle(data)
data_train = data[:100000]
data_valid = data[100000:]

In [127]:
import math 
import numpy 

def inner(x,y):
    return sum([x[i]*y[i] for i in range(len(x))])

def sigmoid(x):
    return 1.0 / (1 + numpy.exp(-x))

In [128]:
len(data)

200000

In [129]:
def feature(datum):
    feat= [1]
    feat.append(sigmoid(datum['helpful']['outOf']))
    feat.append(datum['rating'] * 1.0 / 5.0)
#     feat.append(datum['nWords'])

    if (datum['categoryID'] == 0):
        for i in [1,0,0,0,0]:
            feat.append(i)
    elif (datum['categoryID'] == 1):
        for i in [0,1,0,0,0]:
            feat.append(i)
    elif (datum['categoryID'] == 2):
        for i in [0,0,1,0,0]:
            feat.append(i)
    elif (datum['categoryID'] == 3):
        for i in [0,0,0,1,0]:
            feat.append(i)
    elif (datum['categoryID'] == 4):
        for i in [0,0,0,0,1]:
            feat.append(i)
            
    return feat

In [120]:
X_1 = [feature(d) for d in data if d['helpful']['outOf'] > 2]
X_2 = [feature(d) for d in data if d['helpful']['outOf'] > 0 & d['helpful']['outOf'] < 3]
y_1 = [d['helpful']['nHelpful'] * 1.0 / d['helpful']['outOf'] for d in data if d['helpful']['outOf'] > 2]
y_2 = [d['helpful']['nHelpful'] * 1.0 / d['helpful']['outOf'] for d in data if d['helpful']['outOf'] > 0 & d['helpful']['outOf'] < 3]


In [131]:
X = [feature(d) for d in data if d['helpful']['outOf'] > 0]
y = [d['helpful']['nHelpful'] * 1.0 / d['helpful']['outOf'] for d in data if d['helpful']['outOf'] > 0]

In [133]:
# optimize min
def err1(x):
    error = [abs(d1 - inner(d2, x)) for d1, d2 in zip(y, X)]
    return sum(error)

# def err2(x):
#     error = [abs(d1 - inner(d2, x)) for d1, d2 in zip(y_2, X_2)]
#     return sum(error)

import scipy.optimize
x1 = [0, 0, 0, 0, 0, 0, 0, 0]
# x2 = [0, 0, 0]
theta1 = scipy.optimize.minimize(err1, x1, method="Nelder-Mead").x
# theta2 = scipy.optimize.minimize(err2, x2, method="Nelder-Mead").x
print theta1

[  4.52744703e-01  -3.43388595e-07   3.00288428e-01   2.46967120e-01
   2.46967121e-01  -9.67600570e-02  -4.52574047e-01  -3.00393642e-01]


In [132]:
X[0]

[1, 0.88079707797788231, 0.6, 1, 0, 0, 0, 0]

In [134]:
# MAE
MAE = 0
for d in data_valid:
    if d['helpful']['outOf'] > 0:
        feat = feature(d)
        predict = round(d['helpful']['outOf'] * inner(feat, theta1))
        MAE += math.fabs(predict - d['helpful']['nHelpful'])
#     elif d['helpful']['outOf'] > 0 & d['helpful']['outOf'] < 3:
#         feat = feature(d)
#         predict = round(d['helpful']['outOf'] * inner(feat, theta2))
#         MAE += math.fabs(predict - d['helpful']['nHelpful'])
        
print("Validation MAE = " + str(MAE / len(data_valid)))

Validation MAE = 0.19205


In [ ]:
MAE = 0.17987 sigmoid(outOf)

In [ ]:
MAE = 0.17881 sigmoid(outOf) + rating / 5.0 + theta 1/2(>2, <=2)

In [118]:
# Kaggle
reviews = {}
for l in readGz("assignment1/test_Helpful.json.gz"):
    user,item = l['reviewerID'], l['itemID']
    key = user + ' ' + item
    reviews[key] = {}
    reviews[key]['outOf'] = l['helpful']['outOf']
    reviews[key]['rating'] = l['rating']
    
predictions = open("assignment1/predictions_Helpful.txt", 'w')

for l in open("assignment1/pairs_Helpful.txt"): 
    if l.startswith("userID"):
        # first line
        predictions.write(l)
        continue
    
    u, i, outOf = l.strip().split('-')
    outOf = int(outOf)
    key = u + ' ' + i
    feat = [1, sigmoid(reviews[key]["outOf"]), reviews[key]['rating'] * 1.0 /5.0]
    
    if outOf == 1:
        predict = round(outOf * inner(feat, theta1))
    elif outOf == 2:
        predict = round(outOf * inner(feat, theta2))
    elif outOf > 2:
        predict = round(outOf * inner(feat, theta3))
    predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(predict)+ '\n')

predictions.close ()